In [199]:
import requests
import json
import re
from bs4 import BeautifulSoup as bs4
from IPython.display import display, HTML
import os
import pandas as pd

In [200]:
client_id = os.environ.get("DEVIANTART_CLIENT_ID")
client_secret = os.environ.get("DEVIANTART_CLIENT_SECRET")

In [201]:
token_request = requests.post("https://www.deviantart.com/oauth2/token", params= {
    "client_id": client_id,
    "client_secret": client_secret,
    "grant_type": "client_credentials"
} )

In [202]:
token = token_request.json()["access_token"]

In [203]:
esk_request = requests.get("https://www.deviantart.com/api/v1/oauth2/gallery/all", 
headers={
    'Authorization': 'Bearer {}'.format(token)
    }, 
    params= {
    "username": "esk-masterlist",
    "offset": 250,
    "limit": 24
}).json()["results"]

In [204]:
deviation_ids = [esk["deviationid"] for esk in esk_request]

In [205]:
esk_request = requests.post("https://www.deviantart.com/api/v1/oauth2/deviation/metadata", 
    data={
        "deviationids[]": deviation_ids
    },
    headers={
        'Authorization': 'Bearer {}'.format(token)
    }).json()["metadata"]

In [206]:
def esk_record_from_web_response(raw_esk_record):
    parsed_html = bs4(raw_esk_record["description"])
    bold_texts = parsed_html.body.select('div > b', class_="legacy-journal")
    for bold in bold_texts:
        if re.fullmatch(r"\s+", bold.string) != None:
            continue
        if bold.string == "designer":
            continue
        if bold.string == "ownership":
            break
        tmp = bold.next_sibling.strip()
        if str.endswith(tmp, ".") or str.endswith(tmp, "("):
            tmp = tmp[:-1].strip()
        #if "mutation" in bold.string:    
        tmp = re.sub(r"\(.*\)", "", tmp)
        raw_esk_record[bold.string] = tmp
    raw_esk_record.pop("description", None)
    owner_history = parsed_html.body.find('b', string= "owner history", recursive=True)
    raw_esk_record["owner count"] = len(owner_history.find_next_sibling("sub").findChildren("br")) + 1
    return raw_esk_record


In [207]:
esk_descriptions = [{"title": esk["title"], "description": esk["description"]} for esk in esk_request if (re.fullmatch( r"\d{3,4}", esk["title"]) != None)]

In [208]:
esk_list = []

for esk in esk_descriptions:
    esk_list.append(esk_record_from_web_response(esk))

In [209]:
df = pd.DataFrame(esk_list)